In [1]:
import xgboost as xgb
print(xgb.__version__)

3.1.2


In [2]:
import sys, xgboost as xgb
print(sys.executable)        # should point to .../.venv/bin/python
print(xgb.__version__)       # should print 3.0.4
print(xgb.__file__)          # should live under .../.venv/...

c:\Users\Rajesh\Regression_model_deploy\.venv\Scripts\python.exe
3.1.2
c:\Users\Rajesh\Regression_model_deploy\.venv\Lib\site-packages\xgboost\__init__.py


In [3]:
# ==============================================
# 1. Imports
# ==============================================
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from xgboost import XGBRegressor
import optuna
import mlflow
import mlflow.xgboost

c:\Users\Rajesh\Regression_model_deploy\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# ==============================================
# 2. Load processed datasets
# ==============================================
train_df = pd.read_csv("C://Users//Rajesh//Regression_model_deploy//data//processed//feature_engineered_train.csv")
eval_df  = pd.read_csv("C://Users//Rajesh//Regression_model_deploy//data//processed//feature_engineered_eval.csv")


# Define target + features
target = "price"
X_train, y_train = train_df.drop(columns=[target]), train_df[target]
X_eval, y_eval   = eval_df.drop(columns=[target]), eval_df[target]

print("Train shape:", X_train.shape)
print("Eval shape:", X_eval.shape)

Train shape: (576815, 39)
Eval shape: (148448, 39)


In [5]:
# ==============================================
# 3. Define Optuna objective function with MLflow
# ==============================================
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 200, 1000),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "gamma": trial.suggest_float("gamma", 0.0, 5.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
        "random_state": 42,
        "n_jobs": -1,
        "tree_method": "hist",
    }

    with mlflow.start_run(nested=True):
        model = XGBRegressor(**params)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_eval)
        rmse = float(np.sqrt(mean_squared_error(y_eval, y_pred)))
        mae = float(mean_absolute_error(y_eval, y_pred))
        r2 = float(r2_score(y_eval, y_pred))

        # Log hyperparameters + metrics
        mlflow.log_params(params)
        mlflow.log_metrics({"rmse": rmse, "mae": mae, "r2": r2})

    return rmse

In [6]:
# ==============================================
# 4. Run Optuna study with MLflow
# ==============================================
# Force MLflow to always use the root project mlruns folder
mlflow.set_tracking_uri("file:///C:/Users/Rajesh/Regression_model_deploy/mlruns")
mlflow.set_experiment("xgboost_optuna_housing")

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=15)

print("Best params:", study.best_trial.params)

c:\Users\Rajesh\Regression_model_deploy\.venv\Lib\site-packages\mlflow\tracking\_tracking_service\utils.py:178: FutureWarning: The filesystem tracking backend (e.g., './mlruns') will be deprecated in February 2026. Consider transitioning to a database backend (e.g., 'sqlite:///mlflow.db') to take advantage of the latest MLflow features. See https://github.com/mlflow/mlflow/issues/18534 for more details and migration guidance. For migrating existing data, https://github.com/mlflow/mlflow-export-import can be used.
  return FileStore(store_uri, store_uri)
[I 2026-01-06 13:41:03,226] A new study created in memory with name: no-name-cc791db1-8de7-440b-a007-5a69b35af697
[I 2026-01-06 13:41:17,579] Trial 0 finished with value: 84270.14321653094 and parameters: {'n_estimators': 753, 'max_depth': 4, 'learning_rate': 0.010471902872158793, 'subsample': 0.9262934460256294, 'colsample_bytree': 0.539226804451618, 'min_child_weight': 8, 'gamma': 2.3649175065091743, 'reg_alpha': 0.0017111886001725776

Best params: {'n_estimators': 1000, 'max_depth': 7, 'learning_rate': 0.026074675156751815, 'subsample': 0.7950076412418153, 'colsample_bytree': 0.5135135401950743, 'min_child_weight': 5, 'gamma': 3.344450861353507, 'reg_alpha': 1.09143166146248e-08, 'reg_lambda': 1.013379327801904e-08}


In [7]:
# ==============================================
# 5. Train final model with best params and log to MLflow
# ==============================================
best_params = study.best_trial.params
best_model = XGBRegressor(**best_params)
best_model.fit(X_train, y_train)

y_pred = best_model.predict(X_eval)

mae = mean_absolute_error(y_eval, y_pred)
rmse = np.sqrt(mean_squared_error(y_eval, y_pred))
r2 = r2_score(y_eval, y_pred)

print("Final tuned model performance:")
print("MAE:", mae)
print("RMSE:", rmse)
print("R²:", r2)

with mlflow.start_run(run_name="best_xgboost_model"):
    mlflow.log_params(best_params)
    mlflow.log_metrics({"rmse": rmse, "mae": mae, "r2": r2})

    # FIX: Use mlflow.sklearn to log the XGBRegressor wrapper correctly
    mlflow.sklearn.log_model(best_model, artifact_path="model")

    print("✅ Model logged successfully using mlflow.sklearn")

2026/01/06 13:45:36 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Final tuned model performance:
MAE: 31197.17273646511
RMSE: 68648.93419530994
R²: 0.9635810324926691


2026/01/06 13:45:41 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


✅ Model logged successfully using mlflow.sklearn
